In [2]:
import pandas as pd
import ta
import matplotlib as mpl
%matplotlib inline
# mpl.use('MacOSX')
mpl.interactive(True)
import mplfinance as mpf


In [12]:
df = pd.read_json('AAPL.json')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_values(by='date', inplace=True)

In [13]:
df.head()

,id,symbol,open,high,low,close,volume,split_factor,adj_close,adj_high,adj_low,adj_open
date,,,,,,,,,,,,
2018-03-02 00:00:00+00:00,a92c9f2c-b7f8-4b2b-a392-2be6fbfb61db,AAPL,172.80,176.30,172.45,176.21,38453950,1,41.939318,41.960739,41.044409,41.127712
2018-03-05 00:00:00+00:00,850f750c-b0ac-449e-8817-b6ae57228003,AAPL,175.21,177.74,174.52,176.82,28401366,1,42.084503,42.303470,41.537085,41.701310
2018-03-06 00:00:00+00:00,261ee7f6-8849-4566-93a6-d10457c39820,AAPL,177.91,178.25,176.13,176.67,23788506,1,42.048801,42.424853,41.920277,42.343931
2018-03-07 00:00:00+00:00,cb447269-3b43-4308-b750-a5b4e7b1a8f2,AAPL,174.94,175.85,174.27,175.03,31703462,1,41.658469,41.853635,41.477583,41.637048
2018-03-08 00:00:00+00:00,42a4c183-4d0b-4f73-9b65-34ef288e54a0,AAPL,175.48,177.12,175.07,176.94,23163767,1,42.113063,42.155905,41.667989,41.765572


In [ ]:
df = ta.add_all_ta_features(df, open='open', close='close', high='high', low='low', volume='volume', fillna=True)

In [ ]:
df.head()

In [ ]:
df['signal'] = None
df['limit'] = None

signal_col_idx = df.columns.get_loc('signal')
limit_col_idx = df.columns.get_loc('limit')

for i in range(1, len(df)):
    if df['close'][i] > df['volatility_bbh'][i-1]:
        if df['momentum_rsi'][i] > 70:
            df.iloc[i, signal_col_idx] = 'sell' #sell
            df.iloc[i, limit_col_idx] = df['close'][i] * 0.98 # 2% below the current price
        
    elif df['close'][i] < df['volatility_bbl'][i-1]:
        if df['momentum_rsi'][i] < 30:
            df.iloc[i, signal_col_idx] = 'buy' # Buy
            df.iloc[i, limit_col_idx] = df['close'][i] * 1.02 # 2% above the current price

In [ ]:
buy_signals = df[df['signal'] == 'buy'].index
sell_signals = df[df['signal'] == 'sell'].index
marker_buy = "$BUY$"
marker_sell = "$SELL$"

aps = [mpf.make_addplot(df['volatility_bbh'], color='g'), mpf.make_addplot(df['volatility_bbl'], color='r'), mpf.make_addplot(df['momentum_rsi'], color='b')]
# aps = [mpf.make_addplot(df['volatility_bbh'], color='g'), mpf.make_addplot(df['volatility_bbl'], color='r')]

if buy_signals.size > 0:
    df_buy = df.loc[buy_signals].reindex(df.index)
    aps = aps + [mpf.make_addplot(df_buy['close'], type='scatter', markersize=200, marker=marker_buy, color='g')]

if sell_signals.size > 0:
    df_sell = df.loc[sell_signals].reindex(df.index)
    aps = aps + [mpf.make_addplot(df_sell['close'], type='scatter', markersize=200, marker=marker_sell, color='r')]


mpf.plot(df, type='candle', style = 'yahoo', addplot=aps)

In [1]:
from backtesting import Backtest, Strategy

df['Close'] = df['adj_close']
df['Open'] = df['adj_open']
df['High'] = df['adj_high']
df['Low'] = df['adj_low']
df['Volume'] = df['volume']

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

NameError: name 'df' is not defined

In [34]:
def ta_bollinger_lowerband(values, window, window_dev):
    close = pd.Series(values)
    return ta.volatility.BollingerBands(close=close, window=window, window_dev=window_dev).bollinger_lband()

def ta_bollinger_upperband(values, window, window_dev):
    close = pd.Series(values)
    return ta.volatility.BollingerBands(close=close, window=window, window_dev=window_dev).bollinger_hband()

def ta_momentum_rsi(values, window):
    close = pd.Series(values)
    return ta.momentum.RSIIndicator(close=close, window=window).rsi()


class BollingerRsi(Strategy):
    
    window = 25
    window_dev = 2.5
    rsi_sell = 60
    rsi_buy = 45

    def init(self):
        self.bt_volatility_bbh = self.I(ta_bollinger_upperband, self.data.Close, self.window, self.window_dev)
        self.bt_volatility_bbl = self.I(ta_bollinger_lowerband, self.data.Close, self.window, self.window_dev)
        self.bt_momentum_rsi = self.I(ta_momentum_rsi, self.data.Close, self.window)

    def next(self):
        if (self.data.close > self.bt_volatility_bbh) and (self.bt_momentum_rsi > self.rsi_sell):
            self.sell()

        elif (self.data.close < self.bt_volatility_bbl) and (self.bt_momentum_rsi < self.rsi_buy):
            self.buy()


In [35]:
bt = Backtest(df, BollingerRsi, cash=10_000, commission=.02, exclusive_orders=True)

In [36]:
stats=bt.run()

In [37]:
stats

Start                     2018-03-02 00:00...
End                       2023-03-01 00:00...
Duration                   1825 days 00:00:00
Exposure Time [%]                   42.686804
Equity Final [$]                     80.05717
Equity Peak [$]                       10000.0
Return [%]                         -99.199428
Buy & Hold Return [%]              246.476784
Return (Ann.) [%]                  -61.979887
Volatility (Ann.) [%]                7.744047
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.199428
Avg. Drawdown [%]                  -99.199428
Max. Drawdown Duration     1762 days 00:00:00
Avg. Drawdown Duration     1762 days 00:00:00
# Trades                                  219
Win Rate [%]                         3.652968
Best Trade [%]                      17.783832
Worst Trade [%]                     -7.651495
Avg. Trade [%]                    

In [38]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-231,45,46,43.192761,44.029025,-193.176863,-0.019361,2018-05-07 00:00:00+00:00,2018-05-08 00:00:00+00:00,1 days
1,-227,46,47,43.148444,44.400316,-284.175020,-0.029013,2018-05-08 00:00:00+00:00,2018-05-09 00:00:00+00:00,1 days
2,-218,47,48,43.512310,44.683545,-255.329309,-0.026917,2018-05-09 00:00:00+00:00,2018-05-10 00:00:00+00:00,1 days
3,-211,48,49,43.789875,45.274633,-313.284127,-0.033906,2018-05-10 00:00:00+00:00,2018-05-11 00:00:00+00:00,1 days
4,-201,49,50,44.369141,45.159948,-158.952168,-0.017823,2018-05-11 00:00:00+00:00,2018-05-14 00:00:00+00:00,3 days
...,...,...,...,...,...,...,...,...,...,...
214,-1,575,576,80.166900,86.272972,-6.106072,-0.076167,2020-06-15 00:00:00+00:00,2020-06-16 00:00:00+00:00,1 days
215,-1,576,577,84.547512,87.178757,-2.631245,-0.031121,2020-06-16 00:00:00+00:00,2020-06-17 00:00:00+00:00,1 days
216,-1,577,578,85.435182,86.260698,-0.825516,-0.009662,2020-06-17 00:00:00+00:00,2020-06-18 00:00:00+00:00,1 days
217,-1,578,579,84.535484,87.053567,-2.518083,-0.029787,2020-06-18 00:00:00+00:00,2020-06-19 00:00:00+00:00,1 days


In [39]:
bt.plot()

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p6720', ...)

In [15]:
stats, heatmap = bt.optimize(window=[15, 20, 25, 30, 35], window_dev=[1.0, 1.5, 2.0, 2.5], rsi_sell=[55, 60, 65, 70, 75, 80], rsi_buy=[45, 40, 35, 30], maximize='Return [%]', return_heatmap=True)

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 480 configurations.
  output = _optimize_grid()
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [25]:
bt.plot()

/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/martinlundqvist/trader-app/strategies/.venv/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p6724', ...)

In [ ]:
stats

In [ ]:
heatmap

In [ ]:
heatmap.sort_values().dropna().iloc[-3:]

In [24]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,69,129,179,143.719402,162.113830,1269.215541,0.127988,2022-09-02 00:00:00+00:00,2022-11-14 00:00:00+00:00,73 days
1,-70,179,209,158.871554,139.246083,1373.782970,0.123530,2022-11-14 00:00:00+00:00,2022-12-28 00:00:00+00:00,44 days
2,89,209,251,142.031004,231.880172,7996.575890,0.632602,2022-12-28 00:00:00+00:00,2023-03-01 00:00:00+00:00,63 days


In [17]:
stats['_strategy']

<Strategy BollingerRsi(window=25,window_dev=2.5,rsi_sell=60,rsi_buy=45)>